In [1]:
import librosa
import os
import pandas as pd
import numpy as np

# Set current working directory to parent folder
os.chdir(os.path.abspath(".."))

In [3]:
# Lee el archivo que identifica como los audios estan divididos
df_conv = pd.read_csv(f"data/MSPCORPUS/Time_Labels/conversation_parts.txt", delimiter=";", header=None, names=['Conversation_Part', 'start_time', 'end_time'])

# Creamos dos columnas con el podcast number y la parte de ese podcast
df_conv['PC_Num'] = df_conv['Conversation_Part'].apply(lambda x: x[17:21]).astype(int)
df_conv['Part_Num'] = df_conv['Conversation_Part'].apply(lambda x: x[22:23]).astype(int)

mem = {}

def add_sync_time_columns(row):
    if row.Part_Num == 1:
        st = 0
        mem[row.PC_Num] = row.start_time
    else:
        st = row.start_time - mem[row.PC_Num]
        
    row['m_start_time'] = st
    row['m_end_time'] = row.end_time - mem[row.PC_Num]
    
    return row

# Una fila con los audios en formato inicio: 0 y final: final - inicio
df_conv = df_conv.apply(lambda row: add_sync_time_columns(row), axis=1)
df_conv = df_conv[['Conversation_Part','start_time', 'end_time', 'm_start_time', 'm_end_time', 'PC_Num', 'Part_Num']]
df_conv['Audio_Name'] = df_conv['Conversation_Part'].apply(lambda x: x[0:21]) + ".wav"

In [ ]:
df_annotations = pd.read_excel(f'data/annotations.xlsx')

In [13]:
df_annotations_2 = pd.merge(left = df_annotations[['Name','Emotion','Annotator','PC_Num','Part_Num']], 
                                 right = df_conv[['start_time','end_time','m_start_time','m_end_time','Audio_Name','PC_Num','Part_Num']],
                                 how = 'left', 
                                 on = ['PC_Num','Part_Num'])

In [14]:
df_annotations_2.to_excel('')

,Name,Emotion,Annotator,PC_Num,Part_Num,start_time,end_time,m_start_time,m_end_time,Audio_Name
0,MSP-Conversation_0002_1_007.csv,Valence,7,2,1,77.3370,361.3548,0.0000,284.0178,MSP-Conversation_0002.wav
1,MSP-Conversation_0002_1_009.csv,Valence,9,2,1,77.3370,361.3548,0.0000,284.0178,MSP-Conversation_0002.wav
2,MSP-Conversation_0002_1_010.csv,Valence,10,2,1,77.3370,361.3548,0.0000,284.0178,MSP-Conversation_0002.wav
3,MSP-Conversation_0002_1_013.csv,Valence,13,2,1,77.3370,361.3548,0.0000,284.0178,MSP-Conversation_0002.wav
4,MSP-Conversation_0002_1_015.csv,Valence,15,2,1,77.3370,361.3548,0.0000,284.0178,MSP-Conversation_0002.wav
...,...,...,...,...,...,...,...,...,...,...
12670,MSP-Conversation_2281_4_015.csv,Dominance,15,2281,4,3087.5345,3366.8644,909.3515,1188.6814,MSP-Conversation_2281.wav
12671,MSP-Conversation_2281_4_017.csv,Dominance,17,2281,4,3087.5345,3366.8644,909.3515,1188.6814,MSP-Conversation_2281.wav
12672,MSP-Conversation_2281_4_018.csv,Dominance,18,2281,4,3087.5345,3366.8644,909.3515,1188.6814,MSP-Conversation_2281.wav
12673,MSP-Conversation_2281_4_019.csv,Dominance,19,2281,4,3087.5345,3366.8644,909.3515,1188.6814,MSP-Conversation_2281.wav
